# **fGraph 0 detection from eigenvalues**


<div id="import_libraries">
    <h2>Import Libraries</h2>
</div>


In [1]:
#!pip install pandas
#!pip install scikit-learn
#!pip install matplotlib
#!pip install xgboost

To import the libraries that will be used in this lab, execute the cells below. 


In [ ]:
# Import the libraries we need to use in this lab
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import time
# model trainin set
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
#models 
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBRegressor
#model evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

## Load the dataset




In [2]:
# load the dataset
path= "den_eigen_9.csv"
# read the input data
df = pd.read_csv(path)
df


,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11,DEN_EIGEN12,DEN_EIGEN13
0,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000e+00,-1.000000e+00,-4.142136e-01,-1.915442e-01,2.236068e+00,2.414214,2.611591,5.110693
1,1,-2.361469,-2.297472,-2.000000,-1.966721,-1.124966,-1.000000e+00,-1.000000e+00,-2.220602e-01,-1.674492e-01,2.119400e+00,2.524592,2.528918,4.967228
2,1,-2.386482,-2.325391,-2.000000,-1.912159,-1.285440,-1.000000e+00,-6.830312e-01,-1.797783e-01,-1.155831e-01,2.094682e+00,2.279394,2.692823,4.820966
3,1,-2.386482,-2.307114,-2.000000,-1.857163,-1.356611,-1.285440e+00,-6.458110e-01,-3.103733e-01,-1.155831e-01,2.094682e+00,2.338344,2.692823,5.138728
4,1,-2.612266,-2.112236,-2.000000,-1.802825,-1.288777,-1.116782e+00,-6.584948e-01,-4.018599e-01,-1.133292e-01,1.970475e+00,2.455563,2.713235,4.967297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13967,0,-2.800647,-2.355674,-2.216389,-1.477260,-1.336072,-8.384353e-01,-2.623597e-01,3.669763e-16,3.560807e-01,1.095294e+00,1.290417,3.419570,5.125476
13968,0,-2.618034,-2.230725,-2.181780,-1.618034,-1.059752,-4.828592e-01,-3.819660e-01,-8.894267e-16,1.248329e-18,6.180340e-01,1.441718,3.713585,4.799814
13969,0,-2.461929,-2.000000,-2.000000,-2.000000,-1.168300,-5.639705e-16,-4.498524e-16,-2.664205e-16,-1.408710e-17,2.530236e-16,1.278197,4.000000,4.352033
13970,0,-2.773959,-2.532089,-1.828224,-1.691229,-1.347296,-9.571429e-01,-2.956744e-01,-1.712172e-15,6.496113e-01,8.793852e-01,1.099185,3.457617,5.339815


In [3]:
# Let's round up the columns to 8 decimals.
# Take all non-integer coulumns
cols_to_round = df.columns.difference(['COEFFICIENT_DEN'])

# Round those columns to, say, 8 decimals
df[cols_to_round] = df[cols_to_round].round(8)
df

,COEFFICIENT_DEN,DEN_EIGEN1,DEN_EIGEN2,DEN_EIGEN3,DEN_EIGEN4,DEN_EIGEN5,DEN_EIGEN6,DEN_EIGEN7,DEN_EIGEN8,DEN_EIGEN9,DEN_EIGEN10,DEN_EIGEN11,DEN_EIGEN12,DEN_EIGEN13
0,1,-2.451012,-2.236068,-2.000000,-1.909928,-1.169800,-1.000000,-1.000000,-0.414214,-0.191544,2.236068,2.414214,2.611591,5.110693
1,1,-2.361469,-2.297472,-2.000000,-1.966721,-1.124966,-1.000000,-1.000000,-0.222060,-0.167449,2.119400,2.524592,2.528918,4.967228
2,1,-2.386482,-2.325391,-2.000000,-1.912159,-1.285440,-1.000000,-0.683031,-0.179778,-0.115583,2.094682,2.279394,2.692823,4.820966
3,1,-2.386482,-2.307114,-2.000000,-1.857163,-1.356611,-1.285440,-0.645811,-0.310373,-0.115583,2.094682,2.338344,2.692823,5.138728
4,1,-2.612266,-2.112236,-2.000000,-1.802825,-1.288777,-1.116782,-0.658495,-0.401860,-0.113329,1.970475,2.455563,2.713235,4.967297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13967,0,-2.800647,-2.355674,-2.216389,-1.477260,-1.336072,-0.838435,-0.262360,0.000000,0.356081,1.095294,1.290417,3.419570,5.125476
13968,0,-2.618034,-2.230725,-2.181780,-1.618034,-1.059752,-0.482859,-0.381966,-0.000000,0.000000,0.618034,1.441718,3.713585,4.799814
13969,0,-2.461929,-2.000000,-2.000000,-2.000000,-1.168300,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,1.278197,4.000000,4.352033
13970,0,-2.773959,-2.532089,-1.828224,-1.691229,-1.347296,-0.957143,-0.295674,-0.000000,0.649611,0.879385,1.099185,3.457617,5.339815


# ChatGpt clasifier computation

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


# 1. Extract features and target
y = df["COEFFICIENT_DEN"]
X = df.drop(columns=["COEFFICIENT_DEN"])


In [5]:
# 2. Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [6]:
# 3. Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train a classifier (Logistic Regression here)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_scaled, y_train)

# 5. Predict and evaluate
y_pred = clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7305903398926654
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.79      0.78      1663
           1       0.68      0.64      0.66      1132

    accuracy                           0.73      2795
   macro avg       0.72      0.72      0.72      2795
weighted avg       0.73      0.73      0.73      2795



### Decision tree

In [7]:
# 1. (Optional) scale features – not needed for decision trees but harmless
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Train a Decision Tree classifier
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train_scaled, y_train)

# 3. Predict and evaluate
y_pred = tree_clf.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6994633273703041
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.76      0.75      1663
           1       0.63      0.61      0.62      1132

    accuracy                           0.70      2795
   macro avg       0.69      0.69      0.69      2795
weighted avg       0.70      0.70      0.70      2795



### Random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

# 1. Train the Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=12)
rf_clf.fit(X_train_scaled, y_train)

# 2. Predict and evaluate
y_pred_rf = rf_clf.predict(X_test_scaled)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:\n", classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.7635062611806798
Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.80      0.80      1663
           1       0.71      0.70      0.71      1132

    accuracy                           0.76      2795
   macro avg       0.75      0.75      0.75      2795
weighted avg       0.76      0.76      0.76      2795



In [9]:
# 3. Feature importances from the random forest
importances = rf_clf.feature_importances_
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print("Most important features:")
print(feature_importance.iloc[0:4])

Most important features:
        feature  importance
11  DEN_EIGEN12    0.173925
10  DEN_EIGEN11    0.118394
9   DEN_EIGEN10    0.104672
12  DEN_EIGEN13    0.101248


# Can the model fit the data?

Let's use the 100% of our data to train the model. Does a solution exist? Let's try the logistic regression

In [10]:
# Prepare features and labels
X = df.drop(columns=["COEFFICIENT_DEN"])
y = df["COEFFICIENT_DEN"]

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train on full data
clf = LogisticRegression(max_iter=1000)
clf.fit(X_scaled, y)

# Evaluate on same data
y_pred = clf.predict(X_scaled)
print("Accuracy on full dataset:", accuracy_score(y, y_pred))


Accuracy on full dataset: 0.7369023761809333


In [11]:
from sklearn.svm import SVC

# Train SVM on full dataset
svm_clf = SVC(kernel='rbf')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))


SVM accuracy on full dataset: 0.7870741482965932


Polynomial fit

In [12]:
# Train SVM on full dataset
svm_clf = SVC(kernel='poly')  # Try 'rbf' (default), 'poly', 'linear', etc.
svm_clf.fit(X_scaled, y)

# Evaluate on the same dataset
y_pred_svm = svm_clf.predict(X_scaled)
print("SVM accuracy on full dataset:", accuracy_score(y, y_pred_svm))

SVM accuracy on full dataset: 0.7653879186945319


## Random forest  perfect score

We see that the random forest reach a prefect score.

In [13]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_scaled, y)

y_pred_rf = rf_clf.predict(X_scaled)
print("Random Forest accuracy:", accuracy_score(y, y_pred_rf))

Random Forest accuracy: 0.9997852848554252


In [22]:
importances = rf_clf.feature_importances_
feat_importance = pd.DataFrame({'feature': X.columns, 'importance': importances})
feat_importance = feat_importance.sort_values(by='importance', ascending=False)

print("Top features:\n", feat_importance.head())


Top features:
         feature  importance
10  DEN_EIGEN11    0.185544
9   DEN_EIGEN10    0.124542
11  DEN_EIGEN12    0.113650
8    DEN_EIGEN9    0.090658
5    DEN_EIGEN6    0.078253
